In [ ]:
from chromalab.adaptation import generate_color_wheel

import matplotlib.colors as mcolors
import numpy as np
import polyscope as ps

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [ ]:
# Initialize polyscope and set some scene options
ps.init()

ps.set_build_gui(False)
ps.set_ground_plane_mode("none")
ps.set_SSAA_factor(4)
ps.set_background_color([0, 0, 0])

image_res = 512
ps.set_window_resizable(True)
ps.set_window_size(image_res / 2, image_res / 2)

In [ ]:
def animate_wheel(num_frames, filename, n_rotations=1, fps=60, tetra_image_mode=None):
    """
    Create rotating color wheel animation and save to a .mp4 video file.
    num_frames  -- length of the animation
    filename    -- save name for the .mp4
    n_rotations -- how many rotations to animate
    fps         -- default is 60
    """
    cam_pos = ps.get_view_camera_parameters().get_position()

    if tetra_image_mode is None:
        fd = ps.open_video_file(filename, fps=fps)
    else:
        fd = ps.open_tetra_video_file(filename, fps=fps, save_image_mode=tetra_image_mode)

    thetas = np.linspace(np.pi / 2, np.pi / 2 + (2 * np.pi * n_rotations), num_frames)
    for theta in thetas:
        x = np.cos(theta)
        y = np.sin(theta)
        ps.look_at_dir(cam_pos, [0, 0, 0], [x, y, 0])

        if tetra_image_mode is None:
            ps.write_video_frame(fd)
        else:
            ps.write_tetra_video_frame(fd)

    if tetra_image_mode is None:
        ps.close_video_file(fd)
    else:
        ps.close_tetra_video_file(fd)

In [ ]:
n_channels = 4

if n_channels == 3:
    # Generate some rainbow colors
    hues = np.linspace(0, 0.9, 8)
    colors = np.array([
        mcolors.hsv_to_rgb([hue, 1, 1]) for hue in hues])

    # Create rotating wheel animation
    wheel = generate_color_wheel(colors)
    animate_wheel(num_frames=480, filename='./out/tri/rainbow_wheel_tri.mp4')

In [ ]:
if n_channels == 4:
    # Generate some tetra colors (with random Q)
    hues = np.linspace(0, 0.9, 8)
    colors = np.array([mcolors.hsv_to_rgb([hue, 1, 1]) for hue in hues])
    Qs = np.random.rand(colors.shape[0], 1)
    colors = np.hstack([colors, Qs])

    # Create rotating wheel animation
    wheel = generate_color_wheel(colors)
    animate_wheel(num_frames=480, filename='./out/LMS_Q/rainbow_wheel_tetra.mp4', tetra_image_mode='LMS_Q')